In [9]:
import random
import torch
import matplotlib.pyplot as plt
%matplotlib auto

Using matplotlib backend: Qt5Agg


In [52]:
def synthetic_data(w,b,num_examples):
    x=torch.normal(0,1,(num_examples,len(w)))
    y=torch.matmul(x,w)+b
    y+=torch.normal(0,0.01,y.shape)
    return x,y.reshape((-1,1))

In [53]:
true_w=torch.tensor([2,-3.4])
true_b=4.2
features,labels=synthetic_data(true_w,true_b,1000)

In [54]:
print('features:', features[0],'\nlabel:', labels[0])

features: tensor([-0.2660,  0.2862]) 
label: tensor([2.6816])


In [55]:
plt.scatter(features[:,1].detach().numpy(),labels.detach().numpy(),10)
plt.show()

In [56]:
def data_iter(batch_size,features,labels):
    num_examples=len(features)
    indices=list(range(num_examples))
    random.shuffle(indices)
    for i in range(0,num_examples,batch_size):
        batch_indices=indices[i:min(i+batch_size,num_examples)]
        yield features[batch_indices],labels[batch_indices]

In [57]:
features.shape,labels.shape

(torch.Size([1000, 2]), torch.Size([1000, 1]))

In [58]:
batch_size=10
for x,y in data_iter(batch_size,features,labels):
    print(x.shape)
    print(x.shape,y.shape)
    break

torch.Size([10, 2])
torch.Size([10, 2]) torch.Size([10, 1])


In [59]:
w=torch.normal(0,0.01,size=(2,1),requires_grad=True)
b=torch.zeros(1,requires_grad=True)

In [60]:
def linreg(x,w,b):
    return torch.matmul(x,w)+b

In [61]:
def squared_loss(y_hat,y):
    return (y_hat-y.reshape(y_hat.shape))**2/2

In [62]:
def sgd(params,lr,batch_size):
    with torch.no_grad():
        for p in params:
            p -= lr * p.grad.data /batch_size

In [63]:
lr=0.03
num_epochs=3
net=linreg
loss=squared_loss

In [64]:
type(y)

torch.Tensor

In [65]:
for epoch in range(num_epochs):
    for x,y in data_iter(batch_size,features,labels):
        l=loss(net(x,w,b),y).sum()
        if w.grad is not None:
            for p in [w,b]:
                p.grad.zero_()
        l.backward()
        sgd([w,b],lr,batch_size)
    with torch.no_grad():
        train_l=loss(net(features,w,b),labels)
        print(f'epoch {epoch + 1},loss {float(train_l.mean()):.5f}')
    

epoch 1,loss 0.05524
epoch 2,loss 0.00027
epoch 3,loss 0.00005
